In [1]:
from pathlib import Path
import importlib

import pandas as pd
from thefuzz import fuzz

import config
import module
from tests import test_attendance
from tests import test_session

for p in [module, config, test_attendance, test_session]:
    try:
        importlib.reload(p)  # reload package
    except NameError:
        pass

## Load DF

In [2]:
# if df is not loaded, load df
if "df_ori" not in locals():

    if not config.is_mutiple_files:  # data in one file (data from ken)
        df_ori = pd.read_excel(config.path_raw_data)

    elif config.is_mutiple_files:  # data in multiple files (data from coco)
        files = Path(config.path_raw_data).glob("*.xlsx")
        dfs = [pd.read_excel(file, index_col=None, skiprows=2) for file in files]
        df_ori = pd.concat(dfs, axis=0, ignore_index=True)

    print("df loaded")

# if df is loaded, do not load again
else:
    print("df already exist")

df = df_ori.copy()
print(df.shape)

df loaded
(95211, 14)


# DF Attendance

There are students who change name in the middle of month, e.g. "Devan (PP)" to "Devan (KK)". 

To fix this issue, need to replace the duplicated name with the other name. 

In [3]:
# codes with more than one name
dup_codes = (df
    .groupby(["Service Type", "Student Code"])
    .agg(count=("Student Name", "nunique"))
    .loc[lambda df_: df_["count"] > 1]
    .reset_index()
    ["Student Code"]
    .values
)
# if the set ratio is more than threshold, change the name to the other
for code in dup_codes:
    names = df.loc[df["Student Code"] == code, "Student Name"].unique()
    name1, name2, *name3 = names
    set_ratio = fuzz.token_set_ratio(name1, name2)
    if set_ratio > 0.75:
        df["Student Name"] = df["Student Name"].replace(name1, name2, inplace=False)
        df["Student Name"] = df["Student Name"].replace(name3, name2, inplace=False)

After name replacement is done, we can continue.

In [4]:
df_clean = (
    # note: delete unknown share acc
    module.delete_unknown_shared_acc_teacher(df)
    # drop null rows and cols
    .dropna(how="all", axis="columns")
    .dropna(how="all", axis="rows")
    # clean col name
    .rename(columns=module.map_col)
    .rename(columns=lambda c: c.lower().replace(" ", "_"))
    # obtain current month only
    .assign(class_date=lambda df_: module.convert_to_gmt_plus_7(df_, "class_date"))
    .loc[lambda df_: df_["class_date"].dt.month == module.month]
    # drop duplicated student attendance because i exported the att data multiple times
    # assuming that one student can only exist once at a time
    .drop_duplicates(subset=["student_code", "student_name", "class_time", "class_date"])
    .assign(
        student_name=lambda df_: df_["student_name"].str.upper(),
        # membership = dlx, online or GO
        student_membership=lambda df_: module.create_student_membership(df_),
        # new code = name + code
        student_code=lambda df_: module.create_student_code(df_),
        # clean class type for the first time
        class_type=lambda df_: (
            df_["class_type"]
            .str.replace("Class", "", regex=False)
            .str.title()
            .str.strip()
            .astype("category")
        ),
        # clean student center raw to be used to determine class mode
        student_center=lambda df_: (
            df_["student_center"]
            .str.replace("IN: ", "", regex=False)
            .str.strip()
            .astype("category")
        ),
        # create class time if not exist
        class_time=lambda df_: module.create_class_time(df_),
        # clean class description
        class_description=lambda df_: (df_["class_description"].str.lower().str.strip().astype("str")),
        # clean teacher name for some teachers that are duplicated in coco
        teacher=lambda df_: module.clean_teacher_name(df_).astype("str"),
        # create class duration if not exist
        class_duration=lambda df_: module.create_duration(df_).astype("float"),
        # whether the student attend or not
        student_attendance=lambda df_: module.create_attend(df_),
        # create class location from class description
        class_location=lambda df_: (module.create_class_location_1(df_)),
    )
    # note: may 2023 - replace class with shared account with its real ET
    # this is because of shared account problem
    # should be before merging with df_teacher
    .assign(teacher=lambda df_: module.clean_shared_account_et(df_))
    # merge with df_teacher
    .merge(
        right=module.load_df_teacher(), 
        left_on="teacher",
        right_on="coco_teacher_name",
        how="left", 
        validate="many_to_one"
    )
    # create class mode = offline, online or GOC
    # this is done after teacher merging to get international teacher -> GOC
    .assign(class_mode=lambda df_: module.create_class_mode(df_),)
    # class location 2nd time to get class_location from teacher center
    .assign(class_location=lambda df_: module.create_class_location_2(df_),)
    # assert that online class location is online
    # assign area to each class
    .assign(
        class_location=lambda df_: module.assert_class_location_online(df_),
        class_area=lambda df_: module.create_class_location_area(df_),
    )
    # drop unnecessary columns and sort
    .drop(columns=["student_result", "class_unit"])
    .sort_values(["class_date", "class_time", "student_code"])
    .sort_index(axis=1)  # sort columns alphabetically
    .reset_index(drop=True)
    .assign(index=lambda df_: df_.index + 1)  # create index column
)

# DF Session

Row = single session

In [5]:
df_session = (
    df_clean
    .sort_values(["teacher", "class_date", "class_time", "student_membership"])
    .assign(
        # transform attendance
        # assumes that one teacher can only teach one class at a time
        student_attendance_grouped=lambda df_: (
            df_.groupby(["teacher", "class_date", "class_time", "class_type"])
            ["student_attendance"].transform(lambda x: ", ".join(x))
        ),
        student_membership_grouped=lambda df_: (
            df_.groupby(["teacher", "class_date", "class_time", "class_type"])
            ["student_membership"].transform(lambda x: ", ".join(x))
        ),
    )
    # ! drop column unique to student and drop duplicate
    .drop(
        columns=[
            "student_attendance",
            "student_center",
            "student_code",
            "student_name",
            "student_membership",
            "index",
        ]
    )
    .drop_duplicates(keep="first")
    .assign(
        # create class type grouped
        class_type_grouped=lambda df_: module.create_class_type_grouped(df_),
        # create class service
        class_service=lambda df_: module.create_class_service(df_),
        # the number of people who books this class
        class_booking=lambda df_: module.create_class_booking(df_),
        # the number of people who actually attend
        class_attendance=lambda df_: module.create_class_attendance(df_),
        # delivered or not delivered
        class_status=lambda df_: module.create_class_status(df_),
        # class grouping, which groups each class into its membership type
        class_grouping = lambda df_: df_["class_type_grouped"].map(module.class_grouping)
    )
    # drop unused cols and arrange
    .drop(columns=["student_attendance_grouped", "student_membership_grouped"])
    .sort_index(axis=1)
    .reset_index(drop=True)
    .assign(index=lambda df_: df_.index + 1)
)

# in 2023-09 there is wrong encounter
if config.df_trainer_sheet_name == "2023-09":
    teachers = ["Handayani Khaerunisyah Risma"]
    df_session = df_session.loc[
        ~((df_session["teacher"].isin(teachers)) & (df_session["class_type_grouped"] == "Encounter"))
    ]

/tmp/ipykernel_502062/2786259877.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_.groupby(["teacher", "class_date", "class_time", "class_type"])
/tmp/ipykernel_502062/2786259877.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_.groupby(["teacher", "class_date", "class_time", "class_type"])


# Test

In [6]:
# test to attendance data
test_attendance.test_online_class_is_online_location(df_clean)
test_attendance.test_class_with_online_name_is_online_location(df_clean)
test_attendance.test_class_center_match_with_class_area(df_clean)
test_attendance.test_teacher_center_match_with_teacher_area(df_clean)
test_attendance.test_no_class_time_is_missing(df_clean)
test_attendance.test_shared_account_et_is_mapped(df)
test_attendance.test_shared_account_class_is_mapped(df_clean)
test_attendance.test_goc_class_have_goc_mode(df_clean)
test_attendance.test_teacher_pos_is_complete(df_clean)
test_attendance.test_student_membership_is_mapped(df_clean)

# test to session data
test_session.test_online_class_in_online_location(df_session)
test_session.test_booking_higher_than_eq_attendance(df_session)
test_session.test_vip_class_mapped(df_session)
test_session.test_class_service_mapped(df_session)
test_session.test_class_type_all_filled(df_session)
test_session.test_class_type_grouped_all_filled(df_session)
test_session.test_no_enc_in_class_type_grouped(df_session)

# check duplicated code
test_attendance.test_one_code_is_one_name(df_clean, "student_code", "student_name")

# Save DF

df_clean usually 19000-22000 rows long
df_session usually 4000-5000 rows long

In [7]:
print(f"session = {len(df_session)}")
print(f"attendance = {len(df_clean)}")
print(df_session["class_status"].value_counts(normalize=True))

session = 3773
attendance = 12026
class_status
Given        0.851842
Not Given    0.148158
Name: proportion, dtype: float64


In [8]:
output_folder = Path("output", config.path_raw_data.parts[1], config.path_raw_data.parts[2])
session_filepath = Path(output_folder, f"data-session-{config.path_raw_data.stem}.xlsx")
att_filepath = Path(output_folder, f"data-attendance-{config.path_raw_data.stem}.xlsx")

if not Path(session_filepath).exists() and not Path(att_filepath).exists():
    Path.mkdir(output_folder, exist_ok=True)
    df_session.to_excel(session_filepath, engine="xlsxwriter", index=False)
    df_clean.to_excel(att_filepath, engine="xlsxwriter", index=False)
    print(f"Files saved to {session_filepath} and {att_filepath}")
else:
    print(f"Files already exist in {output_folder}")

Files saved to output/2024/2024-04/data-session-2024-04.xlsx and output/2024/2024-04/data-attendance-2024-04.xlsx


# Experiment

In [9]:
# (
#     df
#     # drop null rows and cols
#     .dropna(how="all", axis="columns")
#     .dropna(how="all", axis="rows")
#     # clean col name
#     .rename(columns=module.map_col)
#     .rename(columns=lambda c: c.lower().replace(" ", "_"))
#     # obtain current month only
#     .assign(class_date=lambda df_: module.convert_to_gmt_plus_7(df_, "class_date"))
#     .loc[lambda df_: df_["class_date"].dt.month == module.month]
#     # drop dup student attendance because i exported the att data multiple x
#     # assuming that one student can only exist once at a time
#     .drop_duplicates(subset=["student_code", "class_time", "class_date"])

#     .loc[lambda df_: df_["student_membership"] == "Standard"]
#     .loc[lambda df_: df_["class_type"] == "Online Encounter"]
#     .loc[lambda df_: ~df_["student_name"].str.lower().str.contains("(go", regex=False)]
#     .loc[lambda df_: ~df_["student_name"].str.lower().str.contains("go)", regex=False)]
# )